In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
# Define the date range
start_date = '2010-01-01'
end_date = '2024-05-13'

In [3]:
# Define the ticker
eur_usd_ticker = 'EURUSD=X'  # Yahoo Finance ticker for EUR/USD exchange rate

In [4]:
#Fetch historical data for EUR/USD exchange rate
try:    
    eur_usd_data = yf.download(eur_usd_ticker, start=start_date, end=end_date)
except Exception as e:
    print("Error fetching data:", e)
    exit()


[*********************100%%**********************]  1 of 1 completed


In [5]:
# Drop any rows with missing data in the interest rate data
eur_usd_data.dropna(inplace=True)

In [6]:
# Display the data
print("EUR/USD Exchange Rate Data:")
eur_usd_data

EUR/USD Exchange Rate Data:


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,0
2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,0
2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,0
2010-01-06,1.436596,1.443460,1.429123,1.440403,1.440403,0
2010-01-07,1.440300,1.444481,1.430206,1.431803,1.431803,0
...,...,...,...,...,...,...
2024-05-06,1.075812,1.079214,1.075639,1.075812,1.075812,0
2024-05-07,1.076751,1.078981,1.075581,1.076751,1.076751,0
2024-05-08,1.075049,1.075847,1.073572,1.075049,1.075049,0


In [7]:
# Create a SQLAlchemy engine
engine = create_engine('sqlite:///financial_data.db')

In [8]:
# Store data in SQL database
eur_usd_data.to_sql('eur_usd_data', engine, if_exists='replace', index=True)

3741

In [9]:
#Verify data stored in SQL database
with engine.connect() as conn:
    result = conn.execute("SELECT * FROM eur_usd_data LIMIT 5").fetchall()
    print("EUR/USD Data from SQL:")
    for row in result:
        print(row)

EUR/USD Data from SQL:
('2010-01-01 00:00:00.000000', 1.4327058792114258, 1.4401957988739014, 1.4327058792114258, 1.4389938116073608, 1.4389938116073608, 0)
('2010-01-04 00:00:00.000000', 1.4310041666030884, 1.4451910257339478, 1.4262080192565918, 1.4423978328704834, 1.4423978328704834, 0)
('2010-01-05 00:00:00.000000', 1.442710041999817, 1.4483097791671753, 1.4351937770843506, 1.4365959167480469, 1.4365959167480469, 0)
('2010-01-06 00:00:00.000000', 1.4365959167480469, 1.4434596300125122, 1.4291226863861084, 1.4404033422470093, 1.4404033422470093, 0)
('2010-01-07 00:00:00.000000', 1.4402996301651, 1.444481372833252, 1.430206060409546, 1.4318032264709473, 1.4318032264709473, 0)


Data Preparation and Cleaning

In [10]:
# Selecting features and target variable
eur_usd_data['Return'] = eur_usd_data['Close'].pct_change()
eur_usd_data.dropna(inplace=True)

In [11]:
eur_usd_data.head()

,Open,High,Low,Close,Adj Close,Volume,Return
Date,,,,,,,
2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,0,0.002366
2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,0,-0.004022
2010-01-06,1.436596,1.443460,1.429123,1.440403,1.440403,0,0.002650
2010-01-07,1.440300,1.444481,1.430206,1.431803,1.431803,0,-0.005971
2010-01-08,1.432090,1.443606,1.427104,1.441109,1.441109,0,0.006499


In [12]:
# Features
X = eur_usd_data[['Open', 'High', 'Low', 'Volume', 'Return']]

In [13]:
# Target variable
y = eur_usd_data['Close']

In [14]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Normalize and standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Initialize the SVM model
svr = SVR(kernel='rbf')

In [17]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5, 0.3]
}

In [18]:
# Initialize GridSearchCV for optimization
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'epsilon': [0.1, 0.2, 0.5, 0.3],
                         'gamma': [0.001, 0.01, 0.1, 1]},
             scoring='r2')

In [19]:
# Best model after optimization
best_svr_model = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'C': 1, 'epsilon': 0.1, 'gamma': 0.1}


In [20]:
# Predict and evaluate the optimized model
y_pred = best_svr_model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print(f"Optimized SVR Model R-squared: {r2}")
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Optimized SVR Model R-squared: 0.9485381974895093
Mean Squared Error: 0.0007295800300014813


In [22]:
# Documenting the optimization process
results = pd.DataFrame(grid_search.cv_results_)
results.to_csv('svm_model_optimization_results.csv', index=False)